- South Korea, Korea 빼고
- 영어 모두 소문자
- , 제거

In [1]:
import re
import unicodedata
from dbConn.mongoConn import config
from dbConn.postgresConn import get_select, get_insert

In [3]:
query = "select pk, name from scrapping.scrapping.instagram_location where is_naver_pk_updated = true;"
eng_name = get_select(query)

In [4]:
len(eng_name)

101802

In [5]:
collection = config(db_name="location_match_db", collection_name="region_name_kor_en")

def unicode_norm(sent):
    normalized = unicodedata.normalize('NFC', sent)
    return normalized

def cleanText(readData):
    text = unicode_norm(readData)
    text = re.sub("[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`'…》]", "", text)
    text = text.lower()
    text = text.replace(',', ' ')
    stopwords = ['우리집', '어딘가', '어딘가에서', 'south Korea', 'korea', 'republic of']
    for stopword in stopwords:
        text = text.replace(stopword, ' ')
    return text

def isEnglishOrKorean(input_s): # 한 단어라도 한글이면 한글, 모두 영어면 영어
    k_count = 0
    e_count = 0
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣'):
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1
    return "k" if k_count>0 else "e"

def isMatched(lang_type='', name=''):
    result = None
    if lang_type == 'k':
        result = collection.find_one({'kor_name': name})
    else:
        result = collection.find_one({'eng_name_lower': name.lower()})
    return result

In [6]:
for (pk, name, ) in eng_name: # eng_name
    name = cleanText(name) # 특수문자 제거, 불용어 제거
    cnt = len(name.split())
    for split_name in name.split():
        kor_or_eng = isEnglishOrKorean(split_name)
        matched_name = isMatched(lang_type=kor_or_eng, name=split_name)
        if matched_name: # 결과 값 있으면
            cnt -= 1
    print(name)
    if cnt < 1: # 행정지역명임
        query = f"""update scrapping.scrapping.instagram_location 
                    set is_naver_pk_updated = false
                    where pk = {pk};"""
        get_insert(query)
    else:
        query = f"""update scrapping.scrapping.instagram_location 
                    set is_naver_pk_updated = true
                    where pk = {pk};"""
        get_insert(query)

bondi beach
update scrapping.scrapping.instagram_location 
                    set is_naver_pk_updated = true
                    where pk = 420211621;
Query push success..
월송정
update scrapping.scrapping.instagram_location 
                    set is_naver_pk_updated = true
                    where pk = 130777393725035;
Query push success..
백빈건널목
update scrapping.scrapping.instagram_location 
                    set is_naver_pk_updated = true
                    where pk = 6750062;
Query push success..
홍룡사 홍룡폭포
update scrapping.scrapping.instagram_location 
                    set is_naver_pk_updated = true
                    where pk = 356076444;
Query push success..
민기남집 무쇠솥 닭매운탕
update scrapping.scrapping.instagram_location 
                    set is_naver_pk_updated = true
                    where pk = 291186923;
Query push success..
공천포바다
update scrapping.scrapping.instagram_location 
                    set is_naver_pk_updated = true
                    where pk = 19343740934